In [9]:
import pandas as pd
import json

from serlo.mysql import db

/home/kulla/.local/share/asdf/installs/python/3.10.1/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


ProgrammingError: 1045 (28000): Access denied for user 'root'@'172.19.6.1' (using password: YES)

In [21]:
def deep_equal(obj1, obj2):
    return json.dumps(obj1, sort_keys=True) == json.dumps(obj2, sort_keys=True)

deep_equal({ "a": { "b": 2 }, "c": [3, 5] }, { "a": { "b": 2 }, "c": [3, 5] })

True

In [48]:
def get_keys(obj):
    def append_key(key):
        return [x + [key] for x in get_keys(obj[key])] 
    
    return sum([ append_key(key) if isinstance(obj[key], dict) else [[key]] for key in obj.keys() ], [])

assert deep_equal(get_keys({ "a": { "b": 2, "r": 4 }, "c": 3 }), [['b', 'a'], ["r", "a"], ["c"]])
assert deep_equal(get_keys({ "a": { "b": { "d": ["a" ]} }, "c": [{"d": 2}] }), [["d", 'b', 'a'], ["c"]])
assert deep_equal(get_keys({}), [])

In [6]:
def get_keys(obj, stack):
    for k, v in obj.items():
        k2 = ([k] if k else []) + stack # don't return empty keys
        if v and isinstance(v, dict):
            for c in get_keys(v, k2):
                yield c
            else: # leaf
                yield k2

def get_values(obj):
    for v in obj.values():
      if not v: continue
      if isinstance(v, dict):
        for c in get_values(v):
          yield c
      else: # leaf
        yield v if isinstance(v, list) else [v]
    
        
def get_value_from_key_array(obj, keys):
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        return get_value_from_key_array(obj[key], keys)

def leaf_equals(keys, wanted_key):
    return keys[0] == wanted_key

In [4]:
def is_plugin(state):
    return "plugin" in state
    
def has_state(plugin):
    return "state" in plugin

In [50]:
# Hier wird nur der Content geholt. Falls dieser ein Youtube-Video ist, könnte description und eventuell reasoning auch interessant sein.
def select_all_entities():
    return pd.read_sql("""
        SELECT entity.id, value
        FROM entity_revision_field, entity
        where field = "content"
            AND entity.current_revision_id = entity_revision_field.entity_revision_id;
        """, db)

In [4]:
# TODO: Inline
def filter_non_trashed(revision):
    return not revision["trashed"]

# TODO: Inline
# get_id(..)
def sortId(revision):
    return revision["id"]

UNREVISED_REVISIONS_QUERY="""
            {
                uuid(id: %s) {
                    __typename
                    title
                    trashed
                    ... on Applet {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                    ... on Article {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Course {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on CoursePage {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Event {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Exercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on ExerciseGroup {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                cohesive
                            }
                        }
                    }
                    ... on GroupedExercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Solution {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Video {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                }
            }
        """

# "id" > umbenennen
def get_newest_unrevised_revision(id, bot):
    try:
        # TODO: "id" als variable
        response = bot.api_call(query=UNREVISED_REVISIONS_QUERY % id)
    except Exception as e:
        print(e)
        print("Some problem with entity id %s" % id)
        raise e
    revisions = response["uuid"]["revisions"]["nodes"]
    # legal_revisions[0] == next(sorted(filter(lambda revision: not revision["trashed"], revisions), key=..., reverse=..))
    legal_revisions = list(filter(lambda revision: not revision["trashed"], revisions))
    legal_revisions.sort(reverse = True, key = sortId)
    return {
        "revision": legal_revisions[0],
        "metadata": {
            "type": response["uuid"]["__typename"],
            "title": response["uuid"]["title"],
            "trashed": response["uuid"]["trashed"]
        }
    }

# rules -> changes
# id -> entityId
def save_revision(id, new_revision, rules, metadata, bot):
    set_entity_input = """{
        changes: {rules},
        subscribeThis: True,
        subscribeThisByEmail: False,
        needsReview: True,
        entityId: id,
        title: metadata["metadata"]["title"],
        content: new_revision
    }"""
    try:
        match metadata["metadata"]["type"]:
            case "Applet":
                custom_input_args={
                    "changes": rules,
                    "entityId": id,
                    "title": metadata["metadata"]["title"],
                    "url": metadata["revision"]["url"]
                }
            case "Article":
                query = """
                    changes: "%s",
                    entityId: %s,
                    title: "%s"
                    content: "%s"
                """ % (rules, str(id), metadata["metadata"]["title"], new_revision)
                response = bot.api_call(query)
            case "Course":
                # TODO: Weitere Ersetzungenb
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setCourse(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: %s,
                                    content: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            case "CoursePage":
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setCoursePage(input: {
                                    changes: '%s',
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: '%s',
                                    content: '%s'
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            case "Event":
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setEvent(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    title: %s,
                                    content: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), metadata["metadata"]["title"], new_revision))
            case "Exercise":
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setExercise(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            case "ExerciseGroup":
                response = bot.api_call(query="""
                        mutation {
                            entity {
                                setExerciseGroup(input: {
                                    changes: %s,
                                    subscribeThis: True,
                                    subscribeThisByEmail: False,
                                    needsReview: True,
                                    entityId: %s,
                                    content: %s
                                    cohesive: %s
                                }) {
                                    success
                                }
                            }
                        }
                    """ % (rules, str(id), new_revision, metadata["revision"]["cohesive"]))
            case "GroupedExercise":
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setGroupedExercise(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            case "Solution":
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setSolution(input: {
                                changes: %s,
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                content: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), new_revision))
            case "Video":
                response = bot.api_call(query="""
                    mutation {
                        entity {
                            setVideo(input: {
                                changes: %s
                                subscribeThis: True,
                                subscribeThisByEmail: False,
                                needsReview: True,
                                entityId: %s,
                                title: %s,
                                content: %s,
                                url: %s
                            }) {
                                success
                            }
                        }
                    }
                """ % (rules, str(id), metadata["metadata"]["title"], new_revision, metadata["revision"]["url"]))
        input_args = create_graphql_input({
            "subscribeThis": True,
            ...
        } | custoum_input_args)
        query = """
            mutation {
                entity {
                    set%s(input: {
                        subscribeThis: true,
                        subscribeThisByEmail: false,
                        needsReview: true,
                        %s
                    }) {
                        success
                    }
                }
            }
        """ % (metadata["metadata"]["type"], input_args)
        response = bot.api_call(query)
    except Exception as e:
        print(id)
        raise e
    print("saved")

In [62]:
def create_graphql_input(args):
    def to_graphql(value):
        if isinstance(value, bool):
            return "true" if value else "false"
        elif isinstance(value, str):
            return repr(value)
        else:
            return value
    
    return "\n".join(["%s: %s" % (k, to_graphql(v)) for k,v in args.items()])

print(create_graphql_input(subscribeThis=True, foo=3, content=json.dumps({"a": 2})))

subscribeThis: true
foo: 3
content: '{"a": 2}'
